# Comparisons between Reefs

*Notebook designed by **Mark Hammerton** from **AIMS***

In [ ]:
import os
import numpy as np
import pandas as pd
from scipy import stats
from scipy.spatial import cKDTree

import urllib.request
import datetime as dt

import netCDF4
from netCDF4 import Dataset, num2date

from matplotlib import pyplot as plt
%config InlineBackend.figure_format = 'retina'
plt.ion()  # To trigger the interactive inline mode

In this notebook, we use the [AIMS eReefs extraction tool](https://extraction.ereefs.aims.gov.au/) to extract time series data for two locations of interest.  

## Example problem: How strong does the wind need to be to set the direction of the surface ocean currents?

We will investigate the relationship between the strength of the wind and the direction of the surface currents for two locations. We will look at **Myrmidon Reef**, which is on the far outer edge of the GBR and is almost right in the middle of the southern **Eastern Australian Current**. 

Our second location will be **Davies Reef** which is further in the reef matrix, but in a similar sector of the GBR. 

### Analysis method

The East Australian Current (EAC) usually is a strong southward current near Myrmidon and Davies Reefs. During winter months the wind moves in a north eastern direction in the near opposite direction to the EAC. When the wind is low the surface currents are dominated by the EAC. As the wind picks up at some speed the wind overpowers the EAC and the surface current moves in the direction of the wind.

To determine the relation between the wind and the surface currents we will pull out hourly time series wind and current data for our two locations of interest. We will then look at the correlation between the wind and current vectors, where a correlation of 1 indicates they are pointing in the same direction, and -1 indicated they are in opposite directions. 

### Setting up to get the data

To extract the time series data using the extraction tool we need to create a CSV file containing the sites of interest. This file needs to contain the coordinates and names of the sites. To create this I first added my points manually in Google Earth. This was done to simply get the location of Myrmidon and Davies Reefs. Using Google Earth to create your CSV file for the extraction tool is only useful if you don't already know the coordinates of your sites.

The location of the two sites are:

```
name,lat,lon
Myrmidon Reef,-18.265599,147.389028
Davies Reef,-18.822840,147.645209
```

### Extracting the data

The CSV file] was uploaded to the [AIMS extraction tool](https://extraction.ereefs.aims.gov.au/) and the extraction was performed with the following settings:

*Data collection:* GBR1 Hydro (Version 2)
*Variables:* Eastward wind speed (wspeed_u), Northward wind speed (wspeed_v), Northward current (v), Eastward current (u)
*Date range:* 1 January 2019 - 31 December 2019
*Time step:* hourly
*Depths:* -2.35 m

Once the extraction request was submitted the dataset was created after an one hour of processing the data was available for download from [Extraction request: Example dataset: Wind-vs-Current at Davies and Myrmidon Reefs (2019)](https://extraction.ereefs.aims.gov.au/data/2009.c451dc3). 

### Downloading the data

In this notebook we will download the data using scripting. There is no need to re-run the extraction request as each extraction performed by the extraction tool has a permanent public page created for it that can be used to facilitate sharing of the data. 

Let's first create a temporary folder to contain the downloaded data. Note: The temp folder is excluded using the .gitignore so it is not saved to the code repository, which is why we must reproduce it.

In [ ]:
if not os.path.exists('temp'):
    os.makedirs('temp')

Now let's download the data. The file to download is 12.9 MB and so this download might take a little while. To allow us to re-run this script without having to wait for the download each time we first check that the download has not already been done.

In [ ]:
extractionfile = os.path.join('temp','2009.c451dc3-collected.csv')  # Use os.path.join so the script will work cross-platform

if not os.path.exists(extractionfile):
    print("Downloading extraction data ...")
    url = 'https://api.ereefs.aims.gov.au/data-extraction/request/2009.c451dc3/files/2009.c451dc3-collected.csv'
    req = urllib.request.urlretrieve(url, extractionfile)
    print(req)
else:
    print("Skipping redownloading extraction data")

## Reading and cleaning the data

Read the resulting CSV file into a [Pandas data frame](https://pandas.pydata.org/pandas-docs/stable/user_guide/10min.html).

In [ ]:
df = pd.read_csv(extractionfile)
df.head()

Here we can see that data is in [tidy format](https://towardsdatascience.com/whats-tidy-data-how-to-organize-messy-datasets-in-python-with-melt-and-pivotable-functions-5d52daa996c9). That is each observation is in one row per time step, variable, depth and site. For each of these combinations there are summary statistics corresponding to the aggregation process that was applied. Somewhat confusingly we extracted hourly data from the hourly eReefs hydrodynamic model and so there was no aggregation applied. As a result all the stats values are the same and the mean corresponds to the actual extracted data values. Let's clean this up a little by removing the redundant columns. Here we also drop the depth variable as the depth of the wind doesn't match the depth of the surface current and so makes it difficult to align the wind and the current together. Since we are only processing a single depth this is an OK simplification. If we were processing the current at multiple depths then we would need a more complex set of data wraggling.

In [ ]:
df2 = df.drop(columns=['median', 'p5', 'p95', 'lowest','highest','Depth']).rename(columns={"mean": "value"})
df2

Next we need to align the wind and current values for a given time step and site on to the same rows of data. Here we are converting the data from tall format to wide format. 

In [ ]:
df3 = (df2.set_index(["Site Name", "Latitude", "Longitude", "Aggregated Date/Time"])
    .pivot(columns="Variable")['value'].reset_index()
        .rename_axis(None, axis=1)
)
df3

## Correlation

Our aim is to create an index that estimates the correlation of the current and the wind vectors.

The correlation of the current and wind vectors can be estimated based using the [dot product](https://www.mathsisfun.com/algebra/vectors-dot-product.html). An overview of the relationship between correlation and using the dot product is described in [Geometric Interpretation of the Correlation between Two Variables](https://medium.com/@ns2586/geometric-interpretation-of-the-correlation-between-two-variables-4011fb3ea18e). The correlation between the two vectors 'r' is given by:

$r = \cos(\theta) = \frac{a \cdot b}{\|a\| \cdot \|b\|}$

where $a \cdot b$ is the dot product between the tow vectors and $\|a\|$ and $\|b\|$ are the magnitudes of the vectors. The dot product can be calculated using the following.

$a \cdot b = a_{x} \times b_{x} + a_{y} \times b_{y}$

and the magnitude of the vectors:

$\|a\| = \sqrt{a_{x}^2+a_{y}^2}$

$\|b\| = \sqrt{b_{x}^2+b_{y}^2}$



In [ ]:
df3['currentmag'] = np.sqrt(df3['u']**2+df3['v']**2)
df3['windmag'] = np.sqrt(df3['wspeed_u']**2+df3['wspeed_v']**2)
df3['windcurrentcorr'] = (df3['u'] * df3['wspeed_u'] + df3['v'] * df3['wspeed_v'])/(df3['currentmag']*df3['windmag'])
df3.head()

Let's look at the relationship between the wind and current as a function of the wind speed. 

Here we are considering each hourly sample as an independent estimate of the relationship. In reality this is not the case as the longer the wind blows the more effect it will have on the current. This is a limitation of the analysis.

In [ ]:
davies = df3.query('`Site Name` == "Davies Reef"')
myrmidon = df3.query('`Site Name` == "Myrmidon Reef"')
myrmidon.head()


Let's create a scatter plot to see if there is a relationship between the wind and currents.

In [ ]:
fig, ax1 = plt.subplots()
fig.set_size_inches(8, 5)

ax1.set_xlabel('Wind speed (m/s)')
ax1.set_ylabel('Wind-current correlation')

ax1.scatter(myrmidon["windmag"], myrmidon["windcurrentcorr"], color='tab:blue', s=2) 
ax1.scatter(davies["windmag"], davies["windcurrentcorr"], color='tab:red', s=2) 

plt.title('Correlation between wind and surface current (hourly data, 2019)',fontsize=11)
fig.tight_layout()

# plt.savefig("CorrelationWindCurrent.png",dpi=300)
plt.show()
fig.clear()
plt.close(fig)
plt.clf()

This scatter plot shows that the relationship between wind and current is weak. This is not surprising given that we are considering just the hourly samples, with no consideration for how long the wind has been blowing. At low wind conditions the current has an even chance of being aligned with the wind (correlation = 1) as in the opposite direction (correlation = -1), however in high wind we can see that there is much more chance that the currents are aligned with the wind.


To understand this relationship better we want to understand how much the wind nudges the current in its direction. If we bin the wind speeds then collect all the correlation samples in each bin then we can see if they average to zero (indicating that there is no relationship between the wind and current) or there is average alignment. 

In [ ]:
fig, ax1 = plt.subplots()
fig.set_size_inches(8, 5)

ax1.set_xlabel('Wind speed (m/s)')
ax1.set_ylabel('Wind-current correlation')

wind = davies["windmag"]
current = davies["windcurrentcorr"]
bin_means, bin_edges, binnumber = stats.binned_statistic(wind, current, 'mean', bins=20)
ax1.hlines(bin_means, bin_edges[:-1], bin_edges[1:], colors='tab:red', lw=5,
           label='Davies Reef')


wind = myrmidon["windmag"]
current = myrmidon["windcurrentcorr"]
bin_means, bin_edges, binnumber = stats.binned_statistic(wind, current, 'mean', bins=20)
ax1.hlines(bin_means, bin_edges[:-1], bin_edges[1:], colors='tab:blue', lw=5,
           label='Myrmidon Reef')

fig.legend(loc='lower left', bbox_to_anchor=(0.75, 0.1))
plt.title('Mean correlation between wind and surface current (hourly data, 2019)',fontsize=11)
fig.tight_layout()

plt.show()
fig.clear()
plt.close(fig)
plt.clf()

From this we can see that for wind speeds below 8 m/s the surface current direction is unrelated to the wind. Above this wind speed the surface current is increasingly determined by the direction of the wind. By the time the wind is 16 m/s the direction of the surface current is dominated by the wind direction.

*Note: It should be remembered that this analysis is based on the eReefs Hydrodynamic model and as such is not based on real data. The eReefs model has however been tuned to accurately capture the flow dynamics of the GBR and so we would expect the estimates from this analysis to be approximately correct.*